In [1]:
!pip install transformers
%pip install numpy
%pip install tensorflow
!pip install scikit-learn
%pip install pandas

In [1]:
import tensorflow as tf
from tqdm import tqdm
import numpy as np
from transformers import BertTokenizer
from transformers import DistilBertTokenizer, RobertaTokenizer, BertConfig, TFBertModel
from sklearn.model_selection import train_test_split
import pickle
import random
import sys
import math
from scipy import stats
from scipy.spatial import distance
import random
import os

In [2]:
BASE_FOLDER = "data/"
#INPUT_FILE = "hate_norm_combined.pkl"
OUTPUT_FOLDER = "hate_intensity_linear_weights_att/"
OUTPUT_FILE = "hate_int_linear_trans42_ATT"
BERT_MODEL = "distilbert-base-uncased"
MAX_LENGTH = 128
TEST_SIZE = 0.15
SEED = 42

USE_ATT = True

BERT_DROPOUT = 0.2
LSTM_UNITS = 50
DENSE_UNITS = 50
LSTM_DROPOUT = 0.1
DENSE_DROPOUT = 0.2
EPOCHS = 2 #(Default 10)
BATCH_SIZE = 32


def random_seed(SEED):
    random.seed(SEED)
    os.environ['PYTHONHASHSEED'] = str(SEED)
    np.random.seed(SEED)
    tf.random.set_seed(SEED)

random_seed(SEED)

In [4]:
%pip install sentencepiece


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 21.0 MB/s eta 0:00:00


In [3]:
from transformers import TFXLNetModel, XLNetTokenizer, XLNetConfig
import tensorflow as tf
import numpy as np
import random
import os
from tqdm import tqdm

# Other constants
BASE_FOLDER = "data/"
#INPUT_FILE = "hate_norm_combined.pkl"
OUTPUT_FOLDER = "hate_intensity_linear_weights_att/"
OUTPUT_FILE = "hate_int_linear_trans42_ATT"
XLNET_MODEL = "xlnet-base-cased"  # For example, using the base version of XLNet
MAX_LENGTH = 128
TEST_SIZE = 0.2
SEED = 42

USE_ATT = True

XLNET_DROPOUT = 0.2  # Set an appropriate dropout rate for XLNet
LSTM_UNITS = 50
DENSE_UNITS = 50
LSTM_DROPOUT = 0.1
DENSE_DROPOUT = 0.2
EPOCHS = 2  # (Default 10)
BATCH_SIZE = 32

# Set random seed
def random_seed(SEED):
    random.seed(SEED)
    os.environ['PYTHONHASHSEED'] = str(SEED)
    np.random.seed(SEED)
    tf.random.set_seed(SEED)

random_seed(SEED)

# Tokenization function
def tokenize(sentences, tokenizer):
    input_ids, input_masks, input_segments = [], [], []
    for sentence in tqdm(sentences):
        inputs = tokenizer.encode_plus(sentence,
                                       add_special_tokens=True,
                                       max_length=MAX_LENGTH,
                                       pad_to_max_length=True,
                                       return_attention_mask=True,
                                       return_token_type_ids=True)
        input_ids.append(inputs['input_ids'])
        input_masks.append(inputs['attention_mask'])
        input_segments.append(inputs['token_type_ids'])

    return np.asarray(input_ids, dtype='int32'), np.asarray(input_masks, dtype='int32'), np.asarray(input_segments, dtype='int32')

# Define XLNet tokenizer
tokenizer = XLNetTokenizer.from_pretrained(XLNET_MODEL, do_lower_case=True, add_special_tokens=True, max_length=MAX_LENGTH, pad_to_max_length=True)

# Define XLNet model configuration
xlnet_config = XLNetConfig.from_pretrained(XLNET_MODEL, dropout=XLNET_DROPOUT, attention_dropout=XLNET_DROPOUT, output_attentions=True)
xlnet_config.output_hidden_states = False

# Load XLNet model
transformer_model = TFXLNetModel.from_pretrained(XLNET_MODEL, config=xlnet_config)
for layer in transformer_model.layers[:3]:  # Freezing the first 3 layers
    layer.trainable = False

# Define model input layers
input_ids_in = tf.keras.layers.Input(shape=(MAX_LENGTH,), name='input_token', dtype='int32')
input_masks_in = tf.keras.layers.Input(shape=(MAX_LENGTH,), name='masked_token', dtype='int32')

# Embedding layer
embedding_layer = transformer_model(input_ids_in, attention_mask=input_masks_in)[0]

# Bidirectional LSTM and subsequent layers
X = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(LSTM_UNITS, return_sequences=True, dropout=LSTM_DROPOUT, recurrent_dropout=LSTM_DROPOUT, kernel_initializer='normal'))(embedding_layer)
if USE_ATT:
    X = tf.keras.layers.Attention(use_scale=True)([X, X])  # Use attention.
X = tf.keras.layers.GlobalMaxPool1D()(X)
X = tf.keras.layers.Dense(DENSE_UNITS, activation='relu', kernel_initializer='normal')(X)
X = tf.keras.layers.Dropout(DENSE_DROPOUT)(X)
X = tf.keras.layers.Dense(1, activation='linear', kernel_initializer='normal')(X)

# Final model
model = tf.keras.Model(inputs=[input_ids_in, input_masks_in], outputs=X)
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['acc', tf.keras.metrics.RootMeanSquaredError()])
model.summary()


spiece.model:   0%|          | 0.00/798k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/760 [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/565M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer TruncatedNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(
Some layers from the model checkpoint at xlnet-base-cased were not used when initializing TFXLNetModel: ['lm_loss']
- This IS expected if you are initializing TFXLNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFXLNetMo

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_token (InputLayer)    [(None, 128)]                0         []                            
                                                                                                  
 masked_token (InputLayer)   [(None, 128)]                0         []                            
                                                                                                  
 tfxl_net_model (TFXLNetMod  TFXLNetModelOutput(last_hi   1167183   ['input_token[0][0]',         
 el)                         dden_state=(None, 128, 768   36         'masked_token[0][0]']        
                             ),                                                                   
                              mems=((128, None, 768),                                         

In [4]:
import pandas as pd
input_data = pd.read_csv("hate_norm_with_span.csv")

In [5]:


intensity_value = []
hate_sentences = []

for i in range(len(input_data)):
    intensity_value.append(int(input_data['Original_Intensity'][i]))
    hate_sentences.append(input_data['Sentence'][i])
    intensity_value.append(int(input_data['Normalized_Intensity'][i]))
    hate_sentences.append(input_data['Normalized_Sentence'][i])

c = list(zip(intensity_value, hate_sentences))
random.shuffle(c)
intensity_value, hate_sentences = zip(*c)

# X_tr, X_te, y_tr, y_te = train_test_split(hate_sentences,
#                                           intensity_value,
#                                           test_size=TEST_SIZE,
#                                           random_state=1)

# train_input_ids, train_input_masks, train_input_segment = tokenize(
#     X_tr, tokenizer)
# test_input_ids, test_input_masks, test_input_segment = tokenize(
#     X_te, tokenizer)
# y_tr = np.asarray(y_tr)
# y_te = np.asarray(y_te)

In [6]:
# Split data into train and temporary test sets
X_train_temp, X_test_temp, y_train_temp, y_test_temp = train_test_split(
    hate_sentences,
    intensity_value,
    test_size=0.25,  # Size of test set
    random_state=42
)

# Split the temporary test set into validation and test sets
X_val, X_te, y_val, y_te = train_test_split(
    X_test_temp,
    y_test_temp,
    test_size=0.6,  # Split the temporary test set into half validation and half test
    random_state=42
)

In [14]:
import pandas as pd

# Assuming X_train_temp, X_val, X_te are lists of sentences
# and y_train_temp, y_val, y_te are lists of intensity values

# Convert the training data into a pandas DataFrame
train_df = pd.DataFrame({
    'Sentence': X_train_temp,
    'Intensity': y_train_temp
})

# Convert the validation data into a pandas DataFrame
val_df = pd.DataFrame({
    'Sentence': X_val,
    'Intensity': y_val
})

# Convert the test data into a pandas DataFrame
test_df = pd.DataFrame({
    'Sentence': X_te,
    'Intensity': y_te
})

# Save these DataFrames to CSV files
train_df.to_csv('hate_train.csv', index=False)
val_df.to_csv('hate_val.csv', index=False)
test_df.to_csv('hate_test.csv', index=False)


In [13]:
type(X_val)

list

In [4]:
import pandas as pd
train_df = pd.read_csv("hate_train.csv")
val_df = pd.read_csv("hate_val.csv")
test_df = pd.read_csv("hate_test.csv")

In [5]:
X_train_temp = train_df['Sentence']
y_train_temp = train_df['Intensity']
X_val = val_df['Sentence']
y_val = val_df['Intensity']
X_te = test_df['Sentence']
y_te = test_df['Intensity']


In [6]:
# Tokenize and convert train, validation, and test sets
train_input_ids, train_input_masks, train_input_segment = tokenize(X_train_temp, tokenizer)
val_input_ids, val_input_masks, val_input_segment = tokenize(X_val, tokenizer)
test_input_ids, test_input_masks, test_input_segment = tokenize(X_te, tokenizer)

# Convert labels to numpy arrays
y_tr = np.asarray(y_train_temp)
y_val = np.asarray(y_val)
y_te = np.asarray(y_te)

  0%|          | 0/4540 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
100%|██████████| 909/909 [00:00<00:00, 1685.53it/s]


In [7]:
# Train model on train set and validate on validation set
model.fit(
    x=[train_input_ids, train_input_masks],
    y=y_tr,
    epochs=5,
    validation_data=([val_input_ids, val_input_masks], y_val),
    batch_size=BATCH_SIZE
)


Epoch 1/5
142/142 [==============================] - 238s 2s/step - loss: 8.1642 - acc: 0.0066 - root_mean_squared_error: 2.8573 - val_loss: 4.5027 - val_acc: 0.0033 - val_root_mean_squared_error: 2.1220
Epoch 2/5
142/142 [==============================] - 208s 1s/step - loss: 4.1906 - acc: 0.0068 - root_mean_squared_error: 2.0471 - val_loss: 4.5394 - val_acc: 0.0033 - val_root_mean_squared_error: 2.1306
Epoch 3/5
142/142 [==============================] - 204s 1s/step - loss: 3.8834 - acc: 0.0068 - root_mean_squared_error: 1.9706 - val_loss: 4.3470 - val_acc: 0.0033 - val_root_mean_squared_error: 2.0850
Epoch 4/5
142/142 [==============================] - 205s 1s/step - loss: 3.7252 - acc: 0.0068 - root_mean_squared_error: 1.9301 - val_loss: 4.3556 - val_acc: 0.0033 - val_root_mean_squared_error: 2.0870
Epoch 5/5
142/142 [==============================] - 207s 1s/step - loss: 3.7014 - acc: 0.0068 - root_mean_squared_error: 1.9239 - val_loss: 5.0478 - val_acc: 0.0033 - val_root_mean_sq

In [8]:
# Evaluate model on the test set
print("Test set evaluation:")
test_results = model.evaluate(x=[test_input_ids, test_input_masks], y=y_te)
val_results = model.evaluate(x=[val_input_ids, val_input_masks], y=y_val)
print(test_results)





Test set evaluation:
19/19 [==============================] - 8s 398ms/step - loss: 5.0478 - acc: 0.0033 - root_mean_squared_error: 2.2467
[4.815035820007324, 0.0066006602719426155, 2.194319009780884]


In [9]:
# Make predictions on the test set
test_predictions = model.predict(x=[test_input_ids, test_input_masks])
test_predictions = np.array(test_predictions, dtype=np.float).flatten()

# Make predictions on the test set
val_predictions = model.predict(x=[val_input_ids, val_input_masks])
val_predictions = np.array(val_predictions, dtype=np.float).flatten()

29/29 [==============================] - 14s 381ms/step


<ipython-input-9-e46c0180883c>:3: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  test_predictions = np.array(test_predictions, dtype=np.float).flatten()


19/19 [==============================] - 7s 373ms/step


<ipython-input-9-e46c0180883c>:7: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  val_predictions = np.array(val_predictions, dtype=np.float).flatten()


In [10]:
import pandas as pd

# Convert predictions and labels into pandas DataFrames
#train_df = pd.DataFrame({'Prediction': train_predictions, 'Label': y_tr})
val_df = pd.DataFrame({'Prediction': val_predictions, 'Label': y_val})
test_df = pd.DataFrame({'Prediction': test_predictions, 'Label': y_te})

# Save DataFrames as CSV files
#train_df.to_csv('train_predictions.csv', index=False)
val_df.to_csv('val_predictions_xlnet.csv', index=False)
test_df.to_csv('test_predictions_xlnet.csv', index=False)


In [6]:
model.fit(x=[train_input_ids, train_input_masks],
          y=y_tr,
          epochs=5,
          validation_split=0.1,
          batch_size=BATCH_SIZE)

print("TEST split", TEST_SIZE)
results = model.evaluate(x=[test_input_ids, test_input_masks], y=y_te)
print(results)
result = model.predict(x=[test_input_ids, test_input_masks])
result = np.array(result, dtype=np.float)
result = result.flatten()
print("pear", stats.pearsonr(result, y_te))
print("cosine", 1 - distance.cosine(result, y_te))

Epoch 1/5
137/137 [==============================] - 255s 2s/step - loss: 8.3190 - acc: 0.0067 - root_mean_squared_error: 2.8843 - val_loss: 4.3021 - val_acc: 0.0021 - val_root_mean_squared_error: 2.0742
Epoch 2/5
137/137 [==============================] - 209s 2s/step - loss: 4.2257 - acc: 0.0073 - root_mean_squared_error: 2.0556 - val_loss: 4.3500 - val_acc: 0.0021 - val_root_mean_squared_error: 2.0857
Epoch 3/5
137/137 [==============================] - 211s 2s/step - loss: 3.9576 - acc: 0.0073 - root_mean_squared_error: 1.9894 - val_loss: 4.2784 - val_acc: 0.0021 - val_root_mean_squared_error: 2.0684
Epoch 4/5
137/137 [==============================] - 207s 2s/step - loss: 3.8770 - acc: 0.0073 - root_mean_squared_error: 1.9690 - val_loss: 4.7134 - val_acc: 0.0021 - val_root_mean_squared_error: 2.1710
Epoch 5/5
137/137 [==============================] - 200s 1s/step - loss: 3.7923 - acc: 0.0073 - root_mean_squared_error: 1.9474 - val_loss: 5.4600 - val_acc: 0.0021 - val_root_mean_sq

<ipython-input-6-586d0141856e>:11: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  result = np.array(result, dtype=np.float)


In [7]:
# Assuming model.predict is already run as in your code
result = model.predict(x=[test_input_ids, test_input_masks])
result = np.array(result, dtype=np.float).flatten()

# Iterate over individual predictions
for i, prediction in enumerate(result):
    print(f"Prediction for record {i}: {prediction}")

# If you also want to compare with true values
for i, (pred, true) in enumerate(zip(result, y_te)):
    print(f"Record {i}: Prediction - {pred}, True Value - {true}")


38/38 [==============================] - 17s 442ms/step


<ipython-input-7-761a7fc2892a>:3: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  result = np.array(result, dtype=np.float).flatten()


Prediction for record 0: 3.6446588039398193
Prediction for record 1: 4.348603248596191
Prediction for record 2: 5.317370414733887
Prediction for record 3: 3.251410484313965
Prediction for record 4: 3.486953020095825
Prediction for record 5: 3.033088445663452
Prediction for record 6: 5.939804553985596
Prediction for record 7: 4.031256675720215
Prediction for record 8: 3.0529847145080566
Prediction for record 9: 3.305241823196411
Prediction for record 10: 4.769223690032959
Prediction for record 11: 4.5460286140441895
Prediction for record 12: 4.799836158752441
Prediction for record 13: 5.732180118560791
Prediction for record 14: 4.6776957511901855
Prediction for record 15: 4.179903507232666
Prediction for record 16: 4.640873908996582
Prediction for record 17: 5.861502647399902
Prediction for record 18: 3.3473281860351562
Prediction for record 19: 4.810091495513916
Prediction for record 20: 4.1053242683410645
Prediction for record 21: 5.001226425170898
Prediction for record 22: 4.95294952

In [8]:
# Assuming test_texts is a list of the original sentences in the same order as test_input_ids
for i, (text, pred, true) in enumerate(zip(test_texts, result, y_te)):
    print(f"Record {i}: Text - {text}")
    print(f"Prediction - {pred}, True Value - {true}\n")


NameError: ignored